# Import das libs utilizadas

In [136]:
import dlib
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display as ipd
import numpy as np
import pandas as pd
from IPython.display import Audio
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import pickle
from tensorflow.keras import applications
from tensorflow.keras.layers import Input
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.utils import to_categorical

import keras.backend as K

from tqdm import tqdm_notebook


# Lendo a base tratada no notebook de preprocessamento

In [138]:
base = pd.read_csv('base_clean_v2.csv')
base.head()

,dir,key_img,frame,dir_lbl,label,cat_labels,reduced_labels
0,extended-cohn-kanade-images\cohn-kanade-images...,S005001,1,Emotion_labels\Emotion\S005\001\S005_001_00000...,0.0,neutral,0
1,extended-cohn-kanade-images\cohn-kanade-images...,S005001,2,Emotion_labels\Emotion\S005\001\S005_001_00000...,0.0,neutral,0
2,extended-cohn-kanade-images\cohn-kanade-images...,S005001,3,Emotion_labels\Emotion\S005\001\S005_001_00000...,0.0,neutral,0
3,extended-cohn-kanade-images\cohn-kanade-images...,S005001,4,Emotion_labels\Emotion\S005\001\S005_001_00000...,3.0,disgust,-1
4,extended-cohn-kanade-images\cohn-kanade-images...,S005001,5,Emotion_labels\Emotion\S005\001\S005_001_00000...,3.0,disgust,-1


In [139]:
base['reduced_labels'].value_counts()

-1    2691
 1    1124
 0     981
Name: reduced_labels, dtype: int64

In [140]:
# Função para transformar o target da forma como o keras entende
def map_keras_label(x):
    if(x == -1):
        # Negativos são 0
        return 0
    elif(x == 0):
        # Neutros são 1
        return 1
    elif(x==1):
        # Positivos são 2
        return 2
    else:
        # Surprise é 3
        return 3

base['keras_labels'] = base['reduced_labels'].map(map_keras_label)

In [141]:
# Separando base de treinamento e de validação
train_dir, test_dir, y_train, y_test = train_test_split(base['dir'], base['keras_labels'], test_size = 0.2, random_state = 42,
                                                       stratify = base['keras_labels'])

In [142]:
# Definindo funções para ler e preprocessar as imagens no tamanho desejado (100 x 100)
size = 100

def load_image(path):
    img = cv2.resize(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB), (size,size))
#     img = cv2.addWeighted(img,4,cv2.GaussianBlur(img, (0,0), size/40) ,-4 ,128)
    return img

def load_test_image(path):
    img = cv2.resize(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB), (size,size))
    return img

In [143]:
# Defininto array de armazenamento das imagens
images = np.empty((len(train_dir), size, size ,3), dtype = np.uint8)

# Lendo as imagens e armazenando
for i, path in tqdm_notebook(enumerate(train_dir)):
    images[i,:,:,:] = load_image(path)

c:\users\lucas\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [144]:
test_images = np.empty((len(test_dir), size, size, 3), dtype = np.uint8)
for i, path in tqdm_notebook(enumerate(test_dir)):
    test_images[i,:,:, :] = load_test_image(path)

c:\users\lucas\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [145]:
# One hot encoding targets
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Treinando o modelo

In [ ]:
img_width, img_height = 100, 100
batch_size = 16
epochs = 20

file_name = 'vgg_100_100_20ep_da_2.h5'

model = Sequential()

vgg16 = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
# resnet = applications.resnet50.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height,3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
# for layer in resnet.layers:
#     layer.trainable = False

model.add(vgg16)

#Adding custom Layers 
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dense(3, activation="softmax"))


# model.add(resnet)

# #Adding custom Layers 
# model.add(Flatten())
# model.add(Dense(200, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(50, activation="relu"))
# model.add(Dense(3, activation="softmax"))


# compile the model 
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.2,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=20)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.2,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=20)

train_generator = train_datagen.flow(images, y_train, batch_size = 16 , seed = 42)

validation_generator = test_datagen.flow(test_images, y_test, batch_size = 16,seed = 42)

# Save the model according to the conditions  
checkpoint = ModelCheckpoint(file_name, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')


# # Train the model 
# model.fit_generator(
# train_generator,
# steps_per_epoch = nb_train_samples,
# epochs = epochs,
# validation_data = validation_generator,
# nb_val_samples = nb_validation_samples,
# callbacks = [checkpoint, early])


model.fit_generator(generator = train_generator,
                    steps_per_epoch = len(train_generator),
                    epochs = epochs,
                    validation_data = validation_generator,
                    callbacks = [checkpoint, early])


Epoch 1/20
239/240 [============================>.] - ETA: 2s - loss: 1.0189 - accuracy: 0.5482
Epoch 00001: val_accuracy improved from -inf to 0.56250, saving model to vgg_100_100_20ep_da_2.h5
240/240 [==============================] - 605s 3s/step - loss: 1.0190 - accuracy: 0.5480 - val_loss: 0.9450 - val_accuracy: 0.5625
Epoch 2/20
239/240 [============================>.] - ETA: 2s - loss: 0.9436 - accuracy: 0.5715
Epoch 00002: val_accuracy improved from 0.56250 to 0.61458, saving model to vgg_100_100_20ep_da_2.h5
240/240 [==============================] - 633s 3s/step - loss: 0.9425 - accuracy: 0.5719 - val_loss: 0.8514 - val_accuracy: 0.6146
Epoch 3/20
239/240 [============================>.] - ETA: 2s - loss: 0.7772 - accuracy: 0.6796
Epoch 00003: val_accuracy improved from 0.61458 to 0.73958, saving model to vgg_100_100_20ep_da_2.h5
240/240 [==============================] - 640s 3s/step - loss: 0.7768 - accuracy: 0.6799 - val_loss: 0.6386 - val_accuracy: 0.7396
Epoch 4/20
239/2

In [ ]:
plt.figure(figsize=(15,7))
plt.title("Perda de treino e teste")
plt.plot(model.history.history['loss'], label = "Treino")
plt.plot(model.history.history['val_loss'], label = "Teste")
plt.legend()

In [ ]:
plt.figure(figsize=(15,7))
plt.title("Acurácia de treino e teste")
plt.plot(model.history.history['accuracy'],label = "Treino")
plt.plot(model.history.history['val_accuracy'], label = "Teste")
plt.legend()
plt.show()

In [ ]:
# Salvando modelo para testes (porém o ideal é utilizar o que salva automaticamente)
model.save_weights("model.h5")

In [ ]:
# Salvando o modelo em um json para reutilizá-lo
model_json = model.to_json()
with open("model_v2.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
from sklearn import metrics

In [ ]:
y_preditos = model.predict(np.array(test_images, dtype=np.float)).argmax(axis=1)

In [ ]:
metrics.confusion_matrix(y_test.argmax(axis=1), y_preditos)

In [ ]:
metrics.accuracy_score(y_test.argmax(axis=1), y_preditos)

In [ ]:
metrics.f1_score(y_test.argmax(axis=1), y_preditos, average= 'macro')